In [2]:
from pyHegel.commands import *
from pyHegel import instruments
log_path = "./logs/"

### Trying to improve dqd charge stability diagrams

In [1]:
# Loading and setting operation point
bilt_x = instruments.iTest_be214x('', slot=1)
bilt_y = instruments.iTest_be214x('', slot=2)
P1, P2 = (bilt_x.ramp, {"ch":1}), (bilt_x.ramp, {"ch":2})
ST = (bilt_y.ramp, {"ch":1})

# dmm = 
# Ids = dmm.readval

NameError: name 'instruments' is not defined

#### Without any correction:
![](./images/no_correction.jpg)

In [ ]:
#  1: just sweep P1, P2
devs_out = [Ids]
dev1, dev2 = P1, P2
start1, start2 = 0, 0
stop1, stop2 = 2, 2
npts1, npts2 = 101, 101
sweep_multi(dev=[dev1, dev2],
            start=[start1, start2],
            stop=[stop1, stop2],
            npts=[npts1, npts2],
            out=devs_out,
            filename=log_path+"no_correction_%T.txt")


#### With ST(P1, P2) virtual gate:
![](./images/virtual_P1.jpg)

In [ ]:
#  2: sweep P1, P2, with V_SET(P1)
sweep(P1, 0, 1, 101, out=ST)
a_p1_st = 0.1 # 1/slope of ST(P1)

sweep(P2, 0, 1, 101, out=ST)
a_p2_st = 0.1 # 1/slope of ST(P2)

In [ ]:

virtual = instruments.virtual_gates_instr(
    virt_names = ["vP1", "vP2", "vST"],
    real_devs = [P1, P2, ST],
    matrix = [[1, 0, a_p1_st],
              [0, 1, a_p1_st],
              [0, 0, 1]]
)
vP1 = virtual.vP1


devs_out = [Ids]
dev1, dev2 = vP1, P2
start1, start2 = 0, 0
stop1, stop2 = 2, 2
npts1, npts2 = 101, 101
sweep_multi(dev=[dev1, dev2],
            start=[start1, start2],
            stop=[stop1, stop2],
            npts=[npts1, npts2],
            out=devs_out,
            filename=log_path+"_vP1_%T.txt")


In [ ]:

#  3: retroaction loop
def feedback_loop_set(Ids_dev, Vg_dev, Vg_first_point, IdsVg_slope,
                      Vg_minmax=(-2,2)):
    # approximate SET oscillation by a straight line.
    # this function make sure we always stays in the optimal point.
    current_point = Vg_dev.get()
    #correction = 